# Extending the pre-night dashboard with a tab with multiple `bokeh` elements

## Notebook perparation

### Load jupyter extensions

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 1

### Imports

Use `aimport` for `schedview` imports for ease of debugging.

In [ ]:
import importlib
import warnings
from astropy.time import Time
import pandas as pd
import panel as pn
import numpy as np
import param
import bokeh.plotting

In [ ]:
import schedview.collect.opsim
import schedview.app
import schedview.app.prenight

### Further preparation of the notebook

Configure the notebook to show `panel` plots and dashboards:

In [ ]:
pn.extension("terminal")

## Create the extended dashboard app

Here, we create a subclass that adds a tab with two new elements: a bokeh bar plot that shows the total visit time for each value of "note", and a table showing corresponding data.

Write the function to create the plot:

In [ ]:
def create_visit_time_by_activity_bars(visits):
    """Create a bar plot showing how much time was used in each activity.

    Parameters
    ----------
    visits : `pandas.DataFrame`
        The visit data

    Returns
    -------
    visit_time_by_activity_plot : `bokeh.models.plots.Plot`
        The bokeh plot.
    """
    accumulated_time = visits.groupby("note")["visitTime"].sum().reset_index()
    plot = bokeh.plotting.figure(
        title="Accumulated time by activity",
        y_range=accumulated_time.note,
        x_axis_label="Total visit time (seconds)",
        y_axis_label="Activity",
    )
    plot.hbar(right="visitTime", y="note", height=0.8, source=accumulated_time)
    return plot

Now write a function to create the table. The `pn.widgets.Tabulator` provides a nice iterface to `pandas.DataFrames`:

In [ ]:
def create_activity_summary_table(visits):
    note_block = np.where(visits["note"].shift() != visits["note"], 1, 0).cumsum()

    def unique_bands(bands):
        these_bands = ", ".join(b for b in "ugrizy" if b in list(bands))
        return these_bands

    activity_summary = (
        visits.reset_index()
        .groupby(note_block)
        .agg(
            {
                "start_date": "min",
                "note": "first",
                "observationId": "count",
                "visitTime": "sum",
                "filter": unique_bands,
                "airmass": "max",
                "moonDistance": "min",
                "sunAlt": "max",
            }
        )
    )

    titles = {
        "note": "activity",
        "observationId": "# visits",
        "visitTime": "duration (s)",
        "filter": "filters",
        "start_date": "start time (UTC)",
        "airmass": "max airmass",
        "moonDistance": "min moon sep.",
        "sunAlt": "max sun alt",
    }

    table_widget = pn.widgets.Tabulator(
        activity_summary, titles=titles, show_index=False
    )
    return table_widget

Now build a subclass of `schedview.app.prenight.prenight.Prenight` to add this new tab. The elemens in the new subclass should override or add three new elements:

1. Override the `shown_tabs` class member in `Prenight` to include your new plot in the list of tabs. The `objects` argument of `param.ListSelector` includes the names of all tabs which can be included when creating a dashboard, and the `default` argument lists the tabs that will be shown if your subclass is instantiated without the `shown_tabs` argument being set.
2. Create methods that returns an instances of `panel.Pane` what wrap a calls to the functions we made above. Wrap them with `@param.depends` decorators so that the plots will get updated when new visits are loaded.
3. Replace `initialize_tab_contents` from the parent class with an implementation that includes the new plot.

In [ ]:
class PrenightWithActivityTab(schedview.app.prenight.prenight.Prenight):
    # Define which tabs should be shown.
    # The name for your new tab should be included in
    # both the default and objects list to be visible
    # by default.
    shown_tabs = param.ListSelector(
        default=[
            "Azimuth and altitude",
            "Airmass vs. time",
            "Sky maps",
            "Table of visits",
            "Reward plots",
            "Activities",
        ],
        objects=[
            "Azimuth and altitude",
            "Airmass vs. time",
            "Sky maps",
            "Table of visits",
            "Reward plots",
            "Visit explorer",
            "Activities",
        ],
        doc="The names of the tabs to show.",
    )

    # If you plot needs to be update when the data is changed
    # e.g. if a new set of data is loaded, list the member
    # in the @param.depends decorator
    @param.depends(
        "_visits",
    )
    def make_activity_bars(self):
        # The dashboard may not have any loaded visits, for example
        # when first loaded, so handle that situation gracefully
        if self._visits is None:
            return "No visits are loaded"

        return create_visit_time_by_activity_bars(self._visits)

    @param.depends(
        "_visits",
    )
    def make_activity_summary_table(self):
        # The dashboard may not have any loaded visits, for example
        # when first loaded, so handle that situation gracefully
        if self._visits is None:
            return "No visits are loaded"

        return create_activity_summary_table(self._visits)

    def initialize_tab_contents(self):
        # Start with the dictionary with the tabs defined
        # in base class.
        tab_contents = super().initialize_tab_contents()

        # Add your new plot to this dictionary
        new_tab_name = "Activities"
        tab_contents[new_tab_name] = pn.Row(
            pn.param.ParamMethod(self.make_activity_bars, loading_indicator=True),
            pn.param.ParamMethod(
                self.make_activity_summary_table, loading_indicator=True
            ),
        )

        return tab_contents

## Try out our new dashboard

Load some sample data:

In [ ]:
sample_data_dir = importlib.resources.files("schedview").joinpath("data")
sample_opsim_db = str(sample_data_dir.joinpath("sample_opsim.db"))
sample_scheduler_pickle = str(sample_data_dir.joinpath("sample_scheduler.pickle.xz"))

Get the date the sample visits start on:

In [ ]:
def get_sim_start_date(opsim_fname):
    opsim = schedview.collect.opsim.read_opsim(opsim_fname)
    start_mjd = opsim.observationStartMJD.min()
    start_datetime_utc = Time(start_mjd, format="mjd").datetime
    night_date = (
        pd.Timestamp(start_datetime_utc, tz="UTC")
        .tz_convert("Chile/Continental")
        .date()
    )
    return night_date


night_date = get_sim_start_date(sample_opsim_db)
night_date

Actually show our dashboard:

In [ ]:
prenight = PrenightWithActivityTab()
pn_app = prenight.make_app(
    night_date,
    opsim_db=sample_opsim_db,
    scheduler=sample_scheduler_pickle,
)
pn_app